In [1]:
# To find values within a column...
#df.loc[df['column_name'] == some_value]
#x = dfs[0][1]
#x.loc[x["ncli"] == 8151]

In [2]:
import os
import re
import math
import pandas as pd
import numpy as np

#import plotly
#import plotly.plotly as py
#from plotly.graph_objs import *
from plotly.offline import download_plotlyjs, init_notebook_mode, iplot
from plotly.graph_objs import *
init_notebook_mode()

#plotly.tools.set_credentials_file(username='theano', api_key='hkYRYxjyM0aAGA5nmGq1')

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


In [3]:
import matplotlib.pyplot as plt

import matplotlib
matplotlib.style.use('ggplot')

In [4]:
home = os.getcwd()
print(home)
sorted([x for x in os.listdir(home)])
collectl_stats = ['Date', '[CPU]User%', '[CPU]Nice%', '[CPU]Sys%', '[CPU]Wait%',
                               '[CPU]Irq%', '[CPU]Soft%', '[CPU]Steal%', '[CPU]Idle%',
                               '[CPU]Totl%', '[CPU]Intrpt/sec', '[CPU]Ctx/sec', '[CPU]Proc/sec',
                               '[CPU]ProcQue', '[CPU]ProcRun', '[CPU]L-Avg1', '[CPU]L-Avg5',
                               '[CPU]L-Avg15', '[CPU]RunTot', '[CPU]BlkTot']
loaded_files = []


/Users/theano/mountpoint/tsch/mininetpipe/results/2017-12-14-08-59


In [5]:
def load_datfile(fpath):
    # Returns a dataframe
    df = pd.read_csv(fpath, 
                     index_col=False,
                     names=["ncli","throughput"])
    return df


def load_collectl(fname):
    df = pd.read_csv(fname,
                     delim_whitespace=True,
                     comment="#",
                     names=collectl_stats)
    return df

def load_data_and_stats(basename):
    pds = []
    prog = re.compile("%s_u\d+_[_1-9a-z]+.dat$" % basename)
    
    for fname in [x for x in os.listdir(".") if prog.match(x)]:
        df_tput = load_datfile(os.path.join(home, fname))
        
        print("Loading file %s" % fname)
        collectl_fname = [x for x in os.listdir(".") if "%s-server-0" % fname in x]
        if len(collectl_fname) > 0:
            df_stats = load_collectl(collectl_fname[0])
            newdf = pd.concat([df_tput.sum(), df_stats.mean()])
        else:
            newdf = df_tput.sum()
        
        nclients = re.search("(?<=_c)\d+", fname)
        try:
            nclients = int(nclients.group(0))
            if newdf["ncli"] != nclients:
                print("skipping df with %d clients...\n" % newdf["ncli"])
                continue
        except:
            # backwards-compatible (i.e., no ncli in fname)
            try:
                print("checking ncli=%d" % newdf["ncli"])
                pow_ncli = 1/math.log(2, newdf["ncli"])
                if abs(round(pow_ncli) - pow_ncli) > 0.000001:
                    print("skipping df with %d clients...\n" % newdf["ncli"])
                    continue
            except ValueError:
                # Eh we tried, toss this data
                continue
            
        expduration = int(re.search("(?<=_u)\d+", fname).group(0))
        
        newdf["fname"] = fname
        newdf["expduration"] = expduration
        newdf["ncores"] = int(fname.split("cores")[0])        
        pds.append(newdf)
        
    return (basename, pd.concat(pds, axis=1)
                        .transpose()
                        .apply(pd.to_numeric, errors='coerce'))

def load_data_no_sum(basename):
    pds = []
    prog = re.compile("%s_u\d+_[_1-9a-z]+.dat$" % basename)
    
    for fname in [x for x in os.listdir(".") if prog.match(x)]:
        df_tput = load_datfile(os.path.join(home, fname))
        
        expduration = int(re.search("(?<=_u)\d+", fname).group(0))
        
        collectl_fname = [x for x in os.listdir(".") if "%s-server-0" % fname in x]
        
        if len(collectl_fname) > 0:
            df_stats = load_collectl(collectl_fname[0])
            newdf = pd.concat([df_tput, df_stats])
        else:
            newdf = df_tput
        
        newdf["expduration"] = expduration
        newdf["fname"] = fname  # for finding weird data
        newdf["ncores"] = int(fname.split("cores")[0])
        pds.append(newdf)

    return (basename, pd.concat(pds).apply(pd.to_numeric, errors='coerce'))

In [ ]:
dfs = {}


In [ ]:
for exp in [
    "64cores",
    "32cores",
    "16cores2",  # Derp forgot to turn off other cores
    "8cores"
]:
    label, df = load_data_and_stats(exp)
    df = df.fillna(df.mean())
    
    # Remove rows where ncli != 2048
    dfs[exp] = df
    

Loading file 64cores_u80_8.dat
checking ncli=64
Loading file 64cores_u80_163.dat
checking ncli=512
Loading file 64cores_u80_274.dat
checking ncli=4096
Loading file 64cores_u80_164.dat
checking ncli=512
Loading file 64cores_u80_273.dat
checking ncli=4096
Loading file 64cores_u80_357.dat
checking ncli=16380
skipping df with 16380 clients...

Loading file 64cores_u80_77.dat
checking ncli=128
Loading file 64cores_u80_118.dat
checking ncli=256
Loading file 64cores_u80_325.dat
checking ncli=8192
Loading file 64cores_u80_359.dat
checking ncli=16366
skipping df with 16366 clients...

Loading file 64cores_u80_79.dat
checking ncli=128
Loading file 64cores_u80_116.dat
checking ncli=256
Loading file 64cores_u80_1.dat
checking ncli=64
Loading file 64cores_u80_111.dat
checking ncli=256
Loading file 64cores_u80_6.dat
checking ncli=64
Loading file 64cores_u80_322.dat
checking ncli=8192
Loading file 64cores_u80_233.dat
checking ncli=2048
Loading file 64cores_u80_124.dat
checking ncli=256
Loading file 6

checking ncli=8192
Loading file 64cores_u80_134.dat
checking ncli=512
Loading file 64cores_u80_223.dat
checking ncli=2048
Loading file 64cores_u80_148.dat
checking ncli=512
Loading file 64cores_u80_199.dat
checking ncli=1024
Loading file 64cores_u160_24.dat
checking ncli=2048
Loading file 64cores_u80_27.dat
checking ncli=64
Loading file 64cores_u80_289.dat
checking ncli=4096
Loading file 64cores_u80_258.dat
checking ncli=4096
Loading file 64cores_u160_23.dat
checking ncli=2048
Loading file 64cores_u80_133.dat
checking ncli=512
Loading file 64cores_u80_224.dat
checking ncli=2048
Loading file 64cores_u80_287.dat
checking ncli=4096
Loading file 64cores_u80_141.dat
checking ncli=512
Loading file 64cores_u80_256.dat
checking ncli=4096
Loading file 64cores_u80_372.dat
checking ncli=16383
skipping df with 16383 clients...

Loading file 64cores_u80_83.dat
checking ncli=128
Loading file 64cores_u80_52.dat
checking ncli=128
Loading file 64cores_u80_84.dat
checking ncli=128
Loading file 64cores_u

In [ ]:
grouped = pd.concat([df for key, df in dfs.items()]).groupby(["ncores", "ncli"])
agged = grouped.agg([np.std, np.mean])

In [ ]:
data = []
barwidth_scalar = 1/4

idx = pd.IndexSlice
mean = agged.loc[:, idx[:, "mean"]]
std = agged.loc[:, idx[:, "std"]]

barwidth = 0.5
#barwidth = mean.index * barwidth_scalar

# Collectl data
"""
data.append(Bar(
    x = x,
    y = mean[("[CPU]Totl%", "mean")],
    yaxis = 'y2',
    name = 'CPU Totl%',
    width = barwidth,
    opacity=0.6
))
traces.append(Bar(
    x = mean.index.astype(str),
    y = mean[("[CPU]Soft%", "mean")],
    yaxis = 'y2',
    name = 'CPU Soft%',
    width = barwidth,
    opacity=0.6
))
traces.append(Bar(
    x = mean.index.astype(str),
    y = mean[("[CPU]User%", "mean")],
    yaxis = 'y2',
    name = 'CPU User%',
    width = barwidth,
    opacity=0.6
))
"""

categoryarray = [ncli + " clients" for ncli in 
                 mean.index.levels[1].astype(int).astype(str)]

# Throughput
for corecount in mean.index.levels[0]:
    data.append(Scatter(
        x = [ncli + " clients" for ncli in 
             mean.loc[corecount].index.astype(int).astype(str)],
        y = mean.loc[corecount][("throughput", "mean")],
        name = "%s server cores" % corecount,
        error_y = dict(
            type='data',
            array=std.loc[corecount][("throughput", "std")],
            visible=True
        ),
    ))


for corecount in mean.index.levels[0]:
    data.append(Bar(
        x = [ncli + " clients" for ncli in 
             mean.loc[corecount].index.astype(int).astype(str)],
        y = mean[("[CPU]Totl%", "mean")][corecount],
        yaxis = 'y2',
        name = 'CPU Totl%% %d cores' % corecount,
        #width = barwidth,
        opacity=0.4
    ))
    
    
layout = Layout (
    xaxis = {'title': 'number of clients',
            'categoryorder': 'array',
            'categoryarray': categoryarray},
    yaxis = {'title': 'throughput (pps)', 'rangemode': 'tozero'},
    yaxis2 = {'title': 'CPU utilization', 
              'side': 'right', 
              'overlaying': 'y',
              'showgrid': False},
    title = 'Small-packet throughput: 2048 clients',
    #legend = {'x': 0.01, 'y': 1},
    #boxmode = "group"
    barmode = "group"
)

fig = {"data": data, "layout": layout}

iplot(fig)

In [45]:
categoryarray

['64 clients', '128 clients', '256 clients', '2048 clients']

In [177]:
std[("throughput", "std")]/mean[("throughput", "mean")]

ncores  ncli  
8       2048.0    0.003985
16      2048.0    0.149165
32      2048.0    0.200193
64      64.0      0.021534
        128.0     0.133382
        2048.0    0.397674
dtype: float64

In [ ]:
data = []

i = 1
for label, df in grouped:
    data.append(Histogram(x=df["throughput"],
                          nbinsx=4,
                          name="%s cores" % label,
                          opacity=0.6,
                          histnorm="probability"))

layout = Layout (
    xaxis = {'title': 'throughput (pps)'},
    yaxis = {'title': 'frequency'},
    title = 'Throughput frequency for varying core counts',
    showlegend = True,
) 
fig = Figure(data=data, layout=layout)
    
py.iplot(fig, filename="tput_frequency_durations_ncores")